<a href="https://colab.research.google.com/github/Nemczek/checkio_database/blob/main/pyCheckio_class_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project pyCheckio

## Setting up

In [27]:
# importing libraries
import requests
import pandas as pd
import sqlalchemy as db
from collections import Counter

In [8]:
# static variables
BASE_URL = 'https://py.checkio.org/api/group-details/'
GROUP_PROGRESS_API_BASE = 'https://py.checkio.org/api/group-progress/'
GROUP_ACTIVITY_API_BASE = 'https://py.checkio.org/api/group-activity/'
# Here you need to put your personal group token
TOKEN = None

URL_WITH_TOKEN = BASE_URL + TOKEN
PROGRESS_API_WITH_TOKEN = GROUP_PROGRESS_API_BASE + TOKEN
ACTIVITY_API_WITH_TOKEN = GROUP_ACTIVITY_API_BASE + TOKEN

In [9]:
# listing all classes
requests.get(URL_WITH_TOKEN).json()['objects']

[{'name': '2021-2022 Jezyki programowania 1',
  'slug': 'jezyki-programowania-1',
  'default_language': 'en',
  'description_text': '',
  'created_at': '2021-10-06',
  'membership_admission': 'Anyone but the approval is required',
  'is_auto_following_enabled': True,
  'is_email_review_activated': True,
  'is_group_default_filter': False,
  'is_full_access': False,
  'is_leaderboard_enabled': True,
  'is_monthly_leaderboard_default': False,
  'is_revisions_enabled': True,
  'is_member_profiles_visible_for_site': False,
  'is_member_solutions_visible_for_site': False,
  'is_site_solutions_visible_for_members': True,
  'members_count': 25,
  'last_activity': '2023-03-24 12:41',
  'course': 'CheckiO [Easy] (no points for votes)',
  'is_current_group': False},
 {'name': 'stara Języki programowania 1 grupa 2',
  'slug': 'jezyki-programowania-1-grupa-2',
  'default_language': 'en',
  'description_text': '',
  'created_at': '2021-02-26',
  'membership_admission': 'Only by invitation via email

In [10]:
def get_slug(url):
  """
  Returns slag parameter of our current class

  Parameters:
  url (str): URL to our class details

  Returns:
  slug (str): the slug parameter 
  """
  slug = requests.get(url).json()['objects'][4]['slug'] # 4 is the index of our class
  return slug
print(get_slug(URL_WITH_TOKEN))

michal-wojcik-2022-2023


## Activity API

In [13]:
# Using Group Progress API
class_slug = get_slug(URL_WITH_TOKEN)
activity_url_with_slug = f"{ACTIVITY_API_WITH_TOKEN}&slug={class_slug}"

resp = requests.get(activity_url_with_slug).json()['objects']
#requests.get(url_with_slug).json()['objects'][2]['data']
resp[2]

{'username': '126086',
 'createdAt': '2023-01-09',
 'data': {'task': {'imageUrl': 'https://d17mnqrx9pmt3e.cloudfront.net/media/logos/task/normal/password-enabled.png',
   'shortText': 'Verify password by condition\n',
   'subject': 'Acceptable Password I',
   'type': 'task',
   'url': '/mission/acceptable-password-i/'},
  'type': 'implementation',
  'user': {'avatarUrl': 'https://www.gravatar.com/avatar/c13a3f1c3609338a3cc532e0eacc70eb?s=80',
   'level': 4,
   'username': '126086',
   'group': {'name': 'Michal Wojcik 2022-2023',
    'url': '/class/michal-wojcik-2022-2023/',
    'owner': 'MichalRyszardWojcik',
    'slug': 'michal-wojcik-2022-2023'},
   'type': 'user',
   'url': '/user/126086/'},
  'url': '/class/michal-wojcik-2022-2023/solution-history/3510415/'}}

## Progress API

In [14]:
progress_url_with_slug = f"{PROGRESS_API_WITH_TOKEN}&slug={class_slug}"
progress_data = requests.get(progress_url_with_slug).json()['objects']

In [21]:
progress_data[0]

{'title': 'Multiply (Intro)',
 'slug': 'multiply-intro',
 'data': [{'username': 'karol2202',
   'openedAt': '2022-11-19 14:11',
   'startedAt': '2022-11-19 14:12',
   'solvedAt': '2022-11-19 14:13',
   'status': 'published',
   'solutions': []},
  {'username': '117374',
   'openedAt': '2022-10-06 21:33',
   'startedAt': '2022-10-09 12:58',
   'solvedAt': '2022-10-09 12:59',
   'status': 'published',
   'solutions': [{'name': 'First attempt',
     'url': 'https://py.checkio.org/mission/multiply-intro/publications/117374/python-3/first-attempt/',
     'createdAt': '2022-11-21 15:31',
     'votes': 0,
     'comments': 0,
     'isRead': False}]},
  {'username': 'Antoni_Wojcik',
   'openedAt': '2022-10-06 21:53',
   'startedAt': '2022-10-06 21:54',
   'solvedAt': '2022-10-23 19:30',
   'status': 'published',
   'solutions': [{'name': 'Multiply',
     'url': 'https://py.checkio.org/mission/multiply-intro/publications/Antoni_Wojcik/python-3/first/',
     'createdAt': '2022-11-21 14:02',
     

In [ ]:
progress_data[0]['data']

In [ ]:
# Save usernames to list
usernames = []
for i in progress_data[0]['data']:
  if i['username'] not in usernames:
    usernames.append(i['username'])
usernames

In [ ]:
# Save names of tasks to list
tasks = []
for task in progress_data:
  if task['title'] not in tasks:
    tasks.append(task['title'])
tasks

In [39]:
# unique statuses
statuses = []
for task in progress_data:
  for user in task['data']:
    statuses.append(user['status'])
set(statuses)

{'new', 'opened', 'published', 'tried'}

In [43]:
# Get [[task_name, num_of_votes, num_of_comments, num_opened, num_published, num_tried, num_new]]
list_of_tasks = []
for task in progress_data:
  num_of_votes = 0
  num_of_comments = 0
  num_of_tries = 0
  list_of_statuses = []
  for entry in task['data']:
    list_of_statuses.append(entry['status'])
    for solution in entry['solutions']:
      num_of_votes += solution['votes']
      num_of_comments += solution['comments']
  counter_object = Counter(list_of_statuses)
  list_of_tasks.append([task['title'], num_of_votes, num_of_comments,
                        counter_object['opened'], counter_object['published'],
                        counter_object['tried'], counter_object['new']])
list_of_tasks

[['Multiply (Intro)', 46, 13, 0, 42, 2, 5],
 ['Acceptable Password I', 43, 36, 0, 43, 0, 6],
 ['Is Even', 72, 16, 0, 43, 0, 6],
 ['First Word (simplified)', 62, 14, 0, 41, 0, 8],
 ['Number Length', 60, 15, 0, 41, 0, 8],
 ['Backward String', 74, 9, 0, 40, 0, 9],
 ['First Word', 30, 5, 0, 40, 0, 9],
 ['Three Words', 53, 5, 0, 40, 0, 9],
 ['Beginning Zeros', 69, 4, 0, 39, 0, 10],
 ['Between Markers (simplified)', 59, 3, 1, 37, 2, 9],
 ['Max Digit', 59, 3, 0, 39, 0, 10],
 ['Correct Sentence', 21, 0, 0, 38, 0, 11],
 ['End Zeros', 46, 8, 0, 38, 0, 11],
 ['Sum Numbers', 49, 5, 3, 38, 0, 8],
 ['Acceptable Password II', 37, 6, 0, 37, 0, 12],
 ['Acceptable Password III', 36, 7, 0, 37, 0, 12],
 ['Acceptable Password IV', 33, 2, 0, 37, 0, 12],
 ['Acceptable Password V', 28, 1, 0, 37, 0, 12],
 ['All Upper I', 57, 6, 0, 37, 0, 12],
 ['Easy Unpack', 32, 5, 0, 37, 0, 12],
 ['Acceptable Password VI', 36, 2, 0, 35, 1, 13],
 ['All the Same', 41, 7, 0, 36, 0, 13],
 ['Backward Each Word', 28, 1, 0, 35, 1, 

In [44]:
# Change list of lists to pandas DataFrame
# Get [[task_name, num_of_votes, num_of_comments, num_opened, num_published, num_tried, num_new]]
task_data = pd.DataFrame(list_of_tasks, columns=['Task', 'Votes', 'Comments',
                                                 'Opened', 'Published', 'Tried',
                                                 'New'])
task_data

,Task,Votes,Comments,Opened,Published,Tried,New
0,Multiply (Intro),46,13,0,42,2,5
1,Acceptable Password I,43,36,0,43,0,6
2,Is Even,72,16,0,43,0,6
3,First Word (simplified),62,14,0,41,0,8
4,Number Length,60,15,0,41,0,8
...,...,...,...,...,...,...,...
206,Transposed Matrix,0,0,1,1,0,47
207,Weak Point,0,0,1,1,0,47
208,What Does the Cow Say?,0,0,1,0,1,47
209,Working Hours Calculator,0,0,2,1,0,46
